In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "4")

In [6]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("../data/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [8]:
df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



### Converting to Spark Types

In [16]:
from pyspark.sql.functions import lit, col
df.select(lit(5), lit("five"), lit(5.0).alias('5.0 float')).show(5)
#-- in SQL SELECT 5, "five", 5.0

+---+----+---------+
|  5|five|5.0 float|
+---+----+---------+
|  5|five|      5.0|
|  5|five|      5.0|
|  5|five|      5.0|
|  5|five|      5.0|
|  5|five|      5.0|
+---+----+---------+
only showing top 5 rows



### Working with Booleans

In [18]:
from pyspark.sql.functions import col
df.where(col("InvoiceNo") != 536365).select("InvoiceNo", "Description").show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



The clenest way for filtering is a singula expression as a string

In [23]:
df.where("InvoiceNo == 536365").show(5, False)
df.where("InvoiceNo <> 536365").show(5, False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

they’re often
easier to understand and to read if you specify them serially. or statements need
to be specified in the same statement

In [24]:
# in Python
from pyspark.sql.functions import instr
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()
# in SQL SELECT * FROM dfTable WHERE StockCode in ("DOT") AND(UnitPrice > 600 OR instr(Description, "POSTAGE") >= 1)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



To filter a DataFrame, you can also just specify a Boolean column:

In [26]:
#withColumn add column
from pyspark.sql.functions import instr
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter |descripFilter)).where("isExpensive").select("unitPrice", "isExpensive").show(5)
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter |descripFilter)).where("isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|isExpensive|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|       true|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|       true|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+



Innstead of creating programmatic step in scala/python for filtering, yu can use sql syntax.

In [27]:
from pyspark.sql.functions import expr
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250")).where("isExpensive").select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



In [58]:
from pyspark.sql.functions import column
df.where('Description is null').show(5)
df.withColumn('boolFromDesc',instr(col("Description"), "POSTAGE") >= 1).where('Description is null').show(5)
df.withColumn('boolFromDesc',instr(col("Description"), "POSTAGE") >= 1).where('Description is null').show(5)
df.na.fill('no-desc',"Description").withColumn('boolFromDesc',instr(col("Description"), "POSTAGE") >= 1).where(instr(col("Description"), "no-desc") >= 1).show(5)
df.withColumn('boolFromDesc',instr(col("Description"), "POSTAGE") >= 1).na.fill('false',"boolFromDesc").where('Description is null').show()


+---------+---------+-----------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+
|   536414|    22139|       null|      56|2010-12-01 11:52:00|      0.0|      null|United Kingdom|
|   536545|    21134|       null|       1|2010-12-01 14:32:00|      0.0|      null|United Kingdom|
|   536546|    22145|       null|       1|2010-12-01 14:33:00|      0.0|      null|United Kingdom|
|   536547|    37509|       null|       1|2010-12-01 14:33:00|      0.0|      null|United Kingdom|
|   536549|   85226A|       null|       1|2010-12-01 14:34:00|      0.0|      null|United Kingdom|
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows

+---------+---------+-----------+--------+-------------------+---------+----------+-

### Working with numbers

In [69]:
from pyspark.sql.functions import expr, pow
from pyspark.sql.functions import lit, round, bround
fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"),fabricatedQuantity.alias("realQuantity")).show(2)

#or in sql version
df.selectExpr("CustomerId","(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)

df.select(expr("CustomerId"),round(fabricatedQuantity).alias("realQuantity")).show(2)

df.select(round(lit(2.5)).alias('testRound'),bround(lit(2.5)).alias('testBround')).show(2)


+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows

+----------+------------+
|CustomerId|realQuantity|
+----------+------------+
|   17850.0|       239.0|
|   17850.0|       419.0|
+----------+------------+
only showing top 2 rows

+---------+----------+
|testRound|testBround|
+---------+----------+
|      3.0|       2.0|
|      3.0|       2.0|
+---------+----------+
only showing top 2 rows



In [72]:
#correlation Pearson

# in Python
from pyspark.sql.functions import corr
df.select(corr("Quantity", "UnitPrice")).show()
df.stat.corr("Quantity", "UnitPrice")

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



-0.04112314436835551

In [74]:
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|               -24|               0.0|           12431.0|     Australia|
|    max|          C

There are a number of statistical functions available in the StatFunctions
Package (accessible using stat as we see in the code block below). These are
DataFrame methods that you can use to calculate a variety of different things.
For instance, you can calculate either exact or approximate quantiles of your
data using the approxQuantile method:

In [75]:
# in Python
colName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice", quantileProbs, relError) # 2.51

[2.51]

freqItems(cols, support=None)
Finding frequent items for columns, possibly with false positives. Using the frequent element count algorithm described in “http://dx.doi.org/10.1145/762471.762473, proposed by Karp, Schenker, and Papadimitriou”. DataFrame.freqItems() and DataFrameStatFunctions.freqItems() are aliases.


Parameters:	
cols – Names of the columns to calculate frequent items for as a list or tuple of strings.
support – The frequency with which to consider an item ‘frequent’. Default is 1%. The support must be greater than 1e-4.


http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions

In [85]:
# in Python
df.stat.crosstab("StockCode", "Quantity").show()
# in Python
df.stat.freqItems(["StockCode", "Quantity"]).show()

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

As a last note, we can also add a unique ID to each row by using the function
monotonically_increasing_id. This function generates a unique value for
each row, starting with 0:

In [87]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()+1).show(2)

+-----------------------------------+
|(monotonically_increasing_id() + 1)|
+-----------------------------------+
|                                  1|
|                                  2|
+-----------------------------------+
only showing top 2 rows



### Working with Strings

In [90]:
from pyspark.sql.functions import initcap, lower, upper
df.select(initcap(col('Description'))).show(2) #initcap, capitalize every first letter
df.select(lower(col('Description'))).show(2)
df.select(upper(col('Description'))).show(2)

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
+--------------------+
only showing top 2 rows

+--------------------+
|  lower(Description)|
+--------------------+
|white hanging hea...|
| white metal lantern|
+--------------------+
only showing top 2 rows

+--------------------+
|  upper(Description)|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
+--------------------+
only showing top 2 rows

